# Wine Quality — Multiple Linear Regression (one command per code cell)
# Steps: load → checks → split → multi-feature OLS → evaluate → CV → persistence (commented) → inference.


# Imports

In [ ]:
import pandas as pd  # data handling


In [ ]:
import numpy as np  # numeric helpers


In [ ]:
from sklearn.model_selection import train_test_split  # splitting utility


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # metrics


In [ ]:
from sklearn.linear_model import LinearRegression  # OLS regressor


# Load dataset selector

In [ ]:
DATASET = 'white'  # 'red' or 'white'


In [ ]:
CSV_PATH = f"/Users/sb/Documents/Study-Material/Machine-Learning/winequality-{DATASET}.csv"  # dataset path


In [ ]:
df = pd.read_csv(CSV_PATH, sep=';')  # read semicolon-separated CSV


# Checks

In [ ]:
df.head()  # preview


In [ ]:
df.shape  # rows, columns


In [ ]:
df.isna().sum()  # missing values per column


# Split

In [ ]:
X = df.drop(columns=['quality'])  # all features


In [ ]:
y = df['quality']  # target


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # split


# Model

In [ ]:
mlr = LinearRegression()  # multiple linear regression model


In [ ]:
mlr.fit(X_train, y_train)  # fit


In [ ]:
pred = mlr.predict(X_test)  # predict


# Evaluate

In [ ]:
mae = mean_absolute_error(y_test, pred)  # MAE


In [ ]:
rmse = mean_squared_error(y_test, pred, squared=False)  # RMSE


In [ ]:
r2 = r2_score(y_test, pred)  # R^2


In [ ]:
{'MAE': mae, 'RMSE': rmse, 'R2': r2}  # metrics


# Cross-validation

In [ ]:
from sklearn.model_selection import cross_validate  # CV helper


In [ ]:
cv = cross_validate(mlr, X, y, cv=5, scoring=('r2','neg_root_mean_squared_error'))  # 5-fold CV


In [ ]:
{k: (np.mean(v), np.std(v)) for k, v in cv.items() if k.startswith('test_')}  # mean±std of test scores


# MLflow Tracking — log params, metrics, and model (install if needed)
# - View UI with: `mlflow ui`

In [ ]:
import mlflow  # experiment tracking


In [ ]:
import mlflow.sklearn  # sklearn logging helpers


In [ ]:
mlflow.set_experiment('MultipleLinearRegression-WineQuality')  # experiment name per algorithm


In [ ]:
mlflow.start_run(run_name=f"{DATASET}")  # begin run


In [ ]:
mlflow.log_params({'fit_intercept': getattr(mlr, 'fit_intercept', None), 'copy_X': getattr(mlr, 'copy_X', None)})  # params


In [ ]:
mlflow.log_metric('MAE', mae)  # metrics


In [ ]:
mlflow.log_metric('RMSE', rmse)  # metrics


In [ ]:
mlflow.log_metric('R2', r2)  # metrics


In [ ]:
mlflow.sklearn.log_model(mlr, artifact_path='model')  # log model


In [ ]:
mlflow.end_run()  # end run


# Persistence — save to disk and reload for inference

In [ ]:
import os  # filesystem utils


In [ ]:
import joblib  # persistence


In [ ]:
from datetime import datetime  # timestamp


In [ ]:
os.makedirs('Machine-Learning/models/MultipleLinearRegression', exist_ok=True)  # ensure dir


In [ ]:
MODEL_PATH = f"Machine-Learning/models/MultipleLinearRegression/multiple_linear_regression_wine_{DATASET}_" + datetime.now().strftime('%Y%m%d_%H%M%S') + ".joblib"  # path


In [ ]:
joblib.dump(mlr, MODEL_PATH)  # save


In [ ]:
loaded = joblib.load(MODEL_PATH)  # load


In [ ]:
float(loaded.predict(X_test.iloc[[0]])[0])  # quick check


# Streamlit App — generate a minimal predictor UI

In [ ]:
APP_CODE = """
import joblib, pandas as pd, streamlit as st
st.set_page_config(page_title='Wine — Multiple Linear Regression', page_icon='🍷')
st.title('🍷 Wine Quality — Multiple Linear Regression')
MODEL_PATH = st.text_input('Model path', value=r'%s')
if MODEL_PATH and st.button('Load model'):
    try:
        st.session_state['model'] = joblib.load(MODEL_PATH)
        st.success('Model loaded')
    except Exception as e:
        st.error(f'Load failed: {e}')
if 'model' in st.session_state:
    cols = %s
    vals = {c: st.number_input(c, value=0.0) for c in cols}
    if st.button('Predict quality'):
        df = pd.DataFrame([vals])
        try:
            y = st.session_state['model'].predict(df)
            st.metric('Predicted quality', f"{float(y[0]):.3f}")
        except Exception as e:
            st.error(f'Predict failed: {e}')
""" % (MODEL_PATH, list(X.columns))


In [ ]:
open('app_streamlit_wine.py', 'w').write(APP_CODE)  # write app file


# AWS S3 Upload (commented stubs)

In [ ]:
# import boto3


In [ ]:
# s3 = boto3.client('s3')


In [ ]:
# bucket = os.environ.get('S3_BUCKET','your-bucket')


In [ ]:
# key = 'wine-models/MultipleLinearRegression/' + os.path.basename(MODEL_PATH)


In [ ]:
# s3.upload_file(MODEL_PATH, bucket, key)


In [ ]:
float(mlr.predict(X_test.iloc[[0]])[0])  # single-row prediction
